# ResNet Training

В этом ноутбуке будет произведено обучение модели `Resnet34` на полученном наборе данных из ноутбука
`./notebooks/01. get_train_data`.

In [3]:
import os
import sys
import numpy as np
import random
from tqdm import tqdm

sys.path.append('.')
from definitions import ROOT_DIR
from detectime.utils import (
    save_checkpoint,
    train_valid_split
)
from detectime.models import load_model
from detectime.loss_function import get_loss
from detectime.optimizers import (
    get_optimizer,
    get_scheduler
)
from detectime.dataset import get_data_loaders
from detectime.train import train, validation

%matplotlib inline
%load_ext autoreload
%autoreload 2

{"asctime": "2021-07-03 02:30:00", "name": "matplotlib.pyplot", "filename": "pyplot.py", "levelname": "DEBUG", "message": "Loaded backend module://matplotlib_inline.backend_inline version unknown."}
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
DATA_PATH = ROOT_DIR / 'data'
NOTEBOOK_PATH = ROOT_DIR / 'notebooks'
INPUT_DATA = DATA_PATH / 'INPUT_DATA'
INPUT_IMAGES_FOLDER = INPUT_DATA / 'TRAIN_DATA'
TRAIN_IMG_FOLDER = INPUT_DATA / 'TRAIN_IMG'
SAVE_TRAIN_IMAGES_HANDS = TRAIN_IMG_FOLDER / 'HANDS'
SAVE_TRAIN_IMAGES_FACES= TRAIN_IMG_FOLDER / 'FACES'

JSON_FOLDER = INPUT_DATA / 'JSON'
FACES_JSON_PRETRAINED = JSON_FOLDER / 'train_with_bboxes.json'
TRAIN_LABELS = INPUT_DATA / 'train.csv'
HAND_DETECTION_FOLDER = ROOT_DIR / 'model' / 'mask_rcnn_hand_detection.h5'
ANNOTATION_DATA = JSON_FOLDER / 'hands.json'

Подгрузим известный нам конфиг и будем использовать `device` для обучения - либо `cpu`, либо `cuda`.

In [44]:
import torch
import yaml
from detectime.utils import convert_dict_to_tuple

CONFIG_PATH = ROOT_DIR / 'config.yml'

with open(CONFIG_PATH) as f:
    data = yaml.safe_load(f)
config = convert_dict_to_tuple(dictionary=data)

device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(f'device: {device_name}')

device: cpu


Обеспечим консистентность и воспроизводимость системы.

In [11]:
seed = config.dataset.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

os.environ['CUDA_VISIBLE_DEVICES'] = config.cuda_id

Подгрузим веса модели `ResNet34`.

In [12]:
print("Loading model...")
net = load_model(config, device=device_name)
print("Done.")

Loading model...
ResNet34
Done.


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\shiro/.cache\torch\hub\checkpoints\resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Подгрузим optimizer, scheduler и loss aggregator.

In [45]:
criterion, criterion_val = get_loss(config, device=device_name)

In [46]:
optimizer = get_optimizer(config, net)

0.002
Opt:  SGD


In [47]:
n_epoch = config.train.n_epoch
scheduler = get_scheduler(config, optimizer)
train_epoch = tqdm(range(config.train.n_epoch),
                   dynamic_ncols=True,
                   desc='Epochs',
                   position=0)

Epochs:   0%|          | 0/21 [00:00<?, ?it/s]

Разделим данные на обучение и валидацию по ключу `video_name`.

In [54]:
train_data, val_data = train_valid_split(
    ANNOTATION_DATA,
    JSON_FOLDER,
    val_size=0.4
)

{"asctime": "2021-07-03 03:03:25", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "55"}
{"asctime": "2021-07-03 03:03:25", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "3"}
{"asctime": "2021-07-03 03:03:25", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "train data length=48"}
{"asctime": "2021-07-03 03:03:25", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "validation data length=7"}
{"asctime": "2021-07-03 03:03:25", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "Savedir train and valid data: C:\\Users\\shiro\\Desktop\\comp\\detectime\\data\\INPUT_DATA\\JSON"}


In [43]:
ANNOTATIONS_TRAIN = JSON_FOLDER / 'train.json'
ANNOTATIONS_VALID= JSON_FOLDER / 'valid.json'

'[{"frame_path":"b89f94cb5b0acaa41c979e6a11b91e29\\/1293d1fb4328d6311892a11b9d36950c.jpg","video_name":"b89f94cb5b0acaa41c979e6a11b91e29","frame_id":116,"label":6,"bbox":[735,452,976,672]},{"frame_path":"b89f94cb5b0acaa41c979e6a11b91e29\\/23260397b76a2d0da2b6e76c593e9018.jpg","video_name":"b89f94cb5b0acaa41c979e6a11b91e29","frame_id":159,"label":5,"bbox":[1306,601,1561,827]},{"frame_path":"b89f94cb5b0acaa41c979e6a11b91e29\\/d04fbd941360010fcdb102a1c81155b6.jpg","video_name":"b89f94cb5b0acaa41c979e6a11b91e29","frame_id":266,"label":6,"bbox":[557,427,750,633]},{"frame_path":"b89f94cb5b0acaa41c979e6a11b91e29\\/46a5e10436ff5996a5273b1e3254c837.jpg","video_name":"b89f94cb5b0acaa41c979e6a11b91e29","frame_id":315,"label":5,"bbox":[494,454,712,733]},{"frame_path":"b89f94cb5b0acaa41c979e6a11b91e29\\/9634382958ae68234098080f5563e393.jpg","video_name":"b89f94cb5b0acaa41c979e6a11b91e29","frame_id":368,"label":5,"bbox":[483,478,634,723]},{"frame_path":"b89f94cb5b0acaa41c979e6a11b91e29\\/227a0137568

In [34]:
dt, dv = get_data_loaders(config,
                          INPUT_IMAGES_FOLDER,
                          ANNOTATIONS_TRAIN,
                          ANNOTATIONS_VALID
                          )

out_dir = str(DATA_PATH / os.path.join(config.outdir, 'test'))
print("Savedir: {}".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

Preparing train reader...
Done.
Preparing valid reader...
Done.
Savedir: C:\Users\shiro\Desktop\comp\detectime\data\experiments\test


C:\Users\shiro\Desktop\detectime\lib\site-packages\torch\utils\data\dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [35]:
for epoch in train_epoch:
    train(net, dt, criterion, optimizer, config, epoch)
    validation(net, dv, criterion_val, epoch)
    save_checkpoint(net, optimizer, scheduler, epoch, out_dir)
    scheduler.step()